In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [133]:
#計測地点のosm_idを指定
start_osm = 204777946
mid_osm = []
end_osm = 1025226214
start_mesh = 50325693243
end_mesh = 50325674423

In [134]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [135]:
getdbname = "kddi_12_new_param"
dbname = "kddi_trkn12_monthly_anbun_3c_25"
trknName = "trkn_12_grp"
sttime = "2022-04-01 12:00:00"

In [136]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        if dis_km == 0:
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値41.9717367226093距離0.8178571707903889 km
存在メッシュ：50325693222値36.5163156224539距離0.37559142325038114 km
存在メッシュ：50325684333値38.608913839224286距離0.5452363503146695 km
存在メッシュ：50325693243値31.883333333333333距離0.0 km
存在メッシュ：50325693241値33.30855016389635距離0.11554095924100155 km
存在メッシュ：50325684331値39.852947072614015距離0.6460889293967952 km
存在メッシュ：50325684321値44.149828877559194距離0.9944330092884479 km
存在メッシュ：50325693223値34.73376606919269距離0.23108184347152202 km
存在メッシュ：50325684332値40.795791323411834距離0.7225244070139163 km
存在メッシュ：50325693224値35.246112515703246距離0.27261727767640936 km
存在メッシュ：50325684312値43.205097442849464距離0.9178445394846149 km
存在メッシュ：50325683444値37.85687921344893距離0.48426962568638426 km
存在メッシュ：50325694111値37.452407928195626距離0.45147952716843076 km
存在メッシュ：50325684323値43.021564033358025距離0.9029656622985835 km
存在メッシュ：50325684141値46.562153183814836距離1.1899978232648964 km
存在メッシュ：50325684122値49.92138893638416距離1.46232

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値40.86286302991215距離0.8178571707903889 km
存在メッシュ：50325693222値35.78176090469948距離0.37559142325038114 km
存在メッシュ：50325684333値37.73077731447679距離0.5452363503146695 km
存在メッシュ：50325693243値31.466666666666665距離0.0 km
存在メッシュ：50325693241値32.794093483719585距離0.11554095924100155 km
存在メッシュ：50325684331値38.889452289325504距離0.6460889293967952 km
存在メッシュ：50325684321値42.89150728564334距離0.9944330092884479 km
存在メッシュ：50325693223値34.121519438992195距離0.23108184347152202 km
存在メッシュ：50325684332値39.7676041017194距離0.7225244070139163 km
存在メッシュ：50325693224値34.59871167959408距離0.27261727767640936 km
存在メッシュ：50325684312値42.011597776820615距離0.9178445394846149 km
存在メッシュ：50325683444値37.03034289060585距離0.48426962568638426 km
存在メッシュ：50325694111値36.65362405087605距離0.45147952716843076 km
存在メッシュ：50325684323値41.84065735262717距離0.9029656622985835 km
存在メッシュ：50325684141値45.13831205729856距離1.1899978232648964 km
存在メッシュ：50325684122値48.267056769779906距離1.46232783

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値41.51083267645481距離0.8178571707903889 km
存在メッシュ：50325693222値36.007231906257275距離0.37559142325038114 km
存在メッシュ：50325684333値38.11831094967034距離0.5452363503146695 km
存在メッシュ：50325693243値31.333333333333332距離0.0 km
存在メッシュ：50325693241値32.77113699731844距離0.11554095924100155 km
存在メッシュ：50325684331値39.373330889604944距離0.6460889293967952 km
存在メッシュ：50325684321値43.708160699620976距離0.9944330092884479 km
存在メッシュ：50325693223値34.208939727865356距離0.23108184347152202 km
存在メッシュ：50325684332値40.324501889900304距離0.7225244070139163 km
存在メッシュ：50325693224値34.72581097315624距離0.27261727767640936 km
存在メッシュ：50325684312値42.755085848707964距離0.9178445394846149 km
存在メッシュ：50325683444値37.35963471374886距離0.48426962568638426 km
存在メッシュ：50325694111値36.95159133154699距離0.45147952716843076 km
存在メッシュ：50325684323値42.56993155992075距離0.9029656622985835 km
存在メッシュ：50325684141値46.14178950042915距離1.1899978232648964 km
存在メッシュ：50325684122値49.53069241657415距離1.462327

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値44.845575168375625距離0.8178571707903889 km
存在メッシュ：50325693222値39.90901205328936距離0.37559142325038114 km
存在メッシュ：50325684333値41.80258598313865距離0.5452363503146695 km
存在メッシュ：50325693243値35.71666666666667距離0.0 km
存在メッシュ：50325693241値37.00633298345331距離0.11554095924100155 km
存在メッシュ：50325684331値42.92830083835272距離0.6460889293967952 km
存在メッシュ：50325684321値46.81651181945801距離0.9944330092884479 km
存在メッシュ：50325693223値38.29599846297418距離0.23108184347152202 km
存在メッシュ：50325684332値43.78147240225401距離0.7225244070139163 km
存在メッシュ：50325693224値38.7596163072351距離0.27261727767640936 km
存在メッシュ：50325684312値45.96163255924512距離0.9178445394846149 km
存在メッシュ：50325683444値41.12207638970605距離0.48426962568638426 km
存在メッシュ：50325694111値40.75607384082195距離0.45147952716843076 km
存在メッシュ：50325684323値45.79555477293901距離0.9029656622985835 km
存在メッシュ：50325684141値48.999403107455635距離1.1899978232648964 km
存在メッシュ：50325684122値52.03914632920994距離1.462327838491

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値41.9717367226093距離0.8178571707903889 km
存在メッシュ：50325693222値36.5163156224539距離0.37559142325038114 km
存在メッシュ：50325684333値38.608913839224286距離0.5452363503146695 km
存在メッシュ：50325693243値31.883333333333333距離0.0 km
存在メッシュ：50325693241値33.30855016389635距離0.11554095924100155 km
存在メッシュ：50325684331値39.852947072614015距離0.6460889293967952 km
存在メッシュ：50325684321値44.149828877559194距離0.9944330092884479 km
存在メッシュ：50325693223値34.73376606919269距離0.23108184347152202 km
存在メッシュ：50325684332値40.795791323411834距離0.7225244070139163 km
存在メッシュ：50325693224値35.246112515703246距離0.27261727767640936 km
存在メッシュ：50325684312値43.205097442849464距離0.9178445394846149 km
存在メッシュ：50325683444値37.85687921344893距離0.48426962568638426 km
存在メッシュ：50325694111値37.452407928195626距離0.45147952716843076 km
存在メッシュ：50325684323値43.021564033358025距離0.9029656622985835 km
存在メッシュ：50325684141値46.562153183814836距離1.1899978232648964 km
存在メッシュ：50325684122値49.92138893638416距離1.46232

...

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値44.845575168375625距離0.8178571707903889 km
存在メッシュ：50325693222値39.90901205328936距離0.37559142325038114 km
存在メッシュ：50325684333値41.80258598313865距離0.5452363503146695 km
存在メッシュ：50325693243値35.71666666666667距離0.0 km
存在メッシュ：50325693241値37.00633298345331距離0.11554095924100155 km
存在メッシュ：50325684331値42.92830083835272距離0.6460889293967952 km
存在メッシュ：50325684321値46.81651181945801距離0.9944330092884479 km
存在メッシュ：50325693223値38.29599846297418距離0.23108184347152202 km
存在メッシュ：50325684332値43.78147240225401距離0.7225244070139163 km
存在メッシュ：50325693224値38.7596163072351距離0.27261727767640936 km
存在メッシュ：50325684312値45.96163255924512距離0.9178445394846149 km
存在メッシュ：50325683444値41.12207638970605距離0.48426962568638426 km
存在メッシュ：50325694111値40.75607384082195距離0.45147952716843076 km
存在メッシュ：50325684323値45.79555477293901距離0.9029656622985835 km
存在メッシュ：50325684141値48.999403107455635距離1.1899978232648964 km
存在メッシュ：50325684122値52.03914632920994距離1.462327838491

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値41.9717367226093距離0.8178571707903889 km
存在メッシュ：50325693222値36.5163156224539距離0.37559142325038114 km
存在メッシュ：50325684333値38.608913839224286距離0.5452363503146695 km
存在メッシュ：50325693243値31.883333333333333距離0.0 km
存在メッシュ：50325693241値33.30855016389635距離0.11554095924100155 km
存在メッシュ：50325684331値39.852947072614015距離0.6460889293967952 km
存在メッシュ：50325684321値44.149828877559194距離0.9944330092884479 km
存在メッシュ：50325693223値34.73376606919269距離0.23108184347152202 km
存在メッシュ：50325684332値40.795791323411834距離0.7225244070139163 km
存在メッシュ：50325693224値35.246112515703246距離0.27261727767640936 km
存在メッシュ：50325684312値43.205097442849464距離0.9178445394846149 km
存在メッシュ：50325683444値37.85687921344893距離0.48426962568638426 km
存在メッシュ：50325694111値37.452407928195626距離0.45147952716843076 km
存在メッシュ：50325684323値43.021564033358025距離0.9029656622985835 km
存在メッシュ：50325684141値46.562153183814836距離1.1899978232648964 km
存在メッシュ：50325684122値49.92138893638416距離1.46232

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値40.86286302991215距離0.8178571707903889 km
存在メッシュ：50325693222値35.78176090469948距離0.37559142325038114 km
存在メッシュ：50325684333値37.73077731447679距離0.5452363503146695 km
存在メッシュ：50325693243値31.466666666666665距離0.0 km
存在メッシュ：50325693241値32.794093483719585距離0.11554095924100155 km
存在メッシュ：50325684331値38.889452289325504距離0.6460889293967952 km
存在メッシュ：50325684321値42.89150728564334距離0.9944330092884479 km
存在メッシュ：50325693223値34.121519438992195距離0.23108184347152202 km
存在メッシュ：50325684332値39.7676041017194距離0.7225244070139163 km
存在メッシュ：50325693224値34.59871167959408距離0.27261727767640936 km
存在メッシュ：50325684312値42.011597776820615距離0.9178445394846149 km
存在メッシュ：50325683444値37.03034289060585距離0.48426962568638426 km
存在メッシュ：50325694111値36.65362405087605距離0.45147952716843076 km
存在メッシュ：50325684323値41.84065735262717距離0.9029656622985835 km
存在メッシュ：50325684141値45.13831205729856距離1.1899978232648964 km
存在メッシュ：50325684122値48.267056769779906距離1.46232783

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値41.51083267645481距離0.8178571707903889 km
存在メッシュ：50325693222値36.007231906257275距離0.37559142325038114 km
存在メッシュ：50325684333値38.11831094967034距離0.5452363503146695 km
存在メッシュ：50325693243値31.333333333333332距離0.0 km
存在メッシュ：50325693241値32.77113699731844距離0.11554095924100155 km
存在メッシュ：50325684331値39.373330889604944距離0.6460889293967952 km
存在メッシュ：50325684321値43.708160699620976距離0.9944330092884479 km
存在メッシュ：50325693223値34.208939727865356距離0.23108184347152202 km
存在メッシュ：50325684332値40.324501889900304距離0.7225244070139163 km
存在メッシュ：50325693224値34.72581097315624距離0.27261727767640936 km
存在メッシュ：50325684312値42.755085848707964距離0.9178445394846149 km
存在メッシュ：50325683444値37.35963471374886距離0.48426962568638426 km
存在メッシュ：50325694111値36.95159133154699距離0.45147952716843076 km
存在メッシュ：50325684323値42.56993155992075距離0.9029656622985835 km
存在メッシュ：50325684141値46.14178950042915距離1.1899978232648964 km
存在メッシュ：50325684122値49.53069241657415距離1.462327

33.8140625000:33.8286458335:132.797656250:132.797656250
存在メッシュ：50325684314値44.845575168375625距離0.8178571707903889 km
存在メッシュ：50325693222値39.90901205328936距離0.37559142325038114 km
存在メッシュ：50325684333値41.80258598313865距離0.5452363503146695 km
存在メッシュ：50325693243値35.71666666666667距離0.0 km
存在メッシュ：50325693241値37.00633298345331距離0.11554095924100155 km
存在メッシュ：50325684331値42.92830083835272距離0.6460889293967952 km
存在メッシュ：50325684321値46.81651181945801距離0.9944330092884479 km
存在メッシュ：50325693223値38.29599846297418距離0.23108184347152202 km
存在メッシュ：50325684332値43.78147240225401距離0.7225244070139163 km
存在メッシュ：50325693224値38.7596163072351距離0.27261727767640936 km
存在メッシュ：50325684312値45.96163255924512距離0.9178445394846149 km
存在メッシュ：50325683444値41.12207638970605距離0.48426962568638426 km
存在メッシュ：50325694111値40.75607384082195距離0.45147952716843076 km
存在メッシュ：50325684323値45.79555477293901距離0.9029656622985835 km
存在メッシュ：50325684141値48.999403107455635距離1.1899978232648964 km
存在メッシュ：50325684122値52.03914632920994距離1.462327838491

In [137]:
connection.close()

In [107]:
#CreateGeojsonFile